# LSTM models on the UCI datasets
- LSTM Models are slow to train. The newly proposed TCNs are shown to be simpler and easier to train, along with better performance

In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot

In [3]:
from keras.utils import to_categorical

# Data loading utilities

In [117]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        #print(data.shape)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [118]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


# Now let us create a MLP network first

In [100]:
# MLP with take 1-D data as an input

# Load the data
trainX, trainy, testX, testy = load_dataset()

trainX = trainX.reshape(7352,-1)
print(trainX.shape)

testX = testX.reshape(2947, -1)
print(testX.shape)

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
(7352, 1152)
(2947, 1152)


In [101]:
# MLP Network
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_54 (Dense)             (None, 64)                73792     
_________________________________________________________________
dense_55 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_56 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_15 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 6)                 198       
Total params: 80,230
Trainable params: 80,230
Non-trainable params: 0
_________________________________________________________________


In [102]:
filepath = 'MLP_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, nb_epoch=100,
            verbose=0, validation_data=(testX, testy),
         callbacks = callbacks)


Epoch 00001: val_acc improved from -inf to 0.75365, saving model to MLP_Best.hdf5

Epoch 00002: val_acc improved from 0.75365 to 0.85748, saving model to MLP_Best.hdf5

Epoch 00003: val_acc did not improve from 0.85748

Epoch 00004: val_acc improved from 0.85748 to 0.87106, saving model to MLP_Best.hdf5

Epoch 00005: val_acc improved from 0.87106 to 0.87784, saving model to MLP_Best.hdf5

Epoch 00006: val_acc did not improve from 0.87784

Epoch 00007: val_acc did not improve from 0.87784

Epoch 00008: val_acc improved from 0.87784 to 0.88157, saving model to MLP_Best.hdf5

Epoch 00009: val_acc did not improve from 0.88157

Epoch 00010: val_acc did not improve from 0.88157

Epoch 00011: val_acc did not improve from 0.88157

Epoch 00012: val_acc did not improve from 0.88157

Epoch 00013: val_acc did not improve from 0.88157

Epoch 00014: val_acc did not improve from 0.88157

Epoch 00015: val_acc improved from 0.88157 to 0.88429, saving model to MLP_Best.hdf5

Epoch 00016: val_acc improv

In [103]:
model.load_weights(filepath)

# Re-evaluate the model
loss, acc = model.evaluate(testX, testy, verbose=2)
print("MLP model, accuracy: {:5.2f}%".format(100 * acc))

2947/2947 - 0s - loss: 0.7471 - acc: 0.8904
MLP model, accuracy: 89.04%


# Let us do a 1-D CNN

In [104]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [105]:
# 1-D CNN Network
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=2, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_13 (Conv1D)           (None, 124, 64)           2944      
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 61, 32)            6176      
_________________________________________________________________
flatten_11 (Flatten)         (None, 1952)              0         
_________________________________________________________________
dense_58 (Dense)             (None, 32)                62496     
_________________________________________________________________
dropout_16 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_59 (Dense)             (None, 6)                 198       
Total params: 71,814
Trainable params: 71,814
Non-trainable params: 0
_________________________________________________

In [106]:
filepath = '1D_CNN_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, nb_epoch=40,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/40
7296/7352 [============================>.] - ETA: 0s - loss: 0.7941 - acc: 0.6720
Epoch 00001: val_acc improved from -inf to 0.81201, saving model to 1D_CNN_Best.hdf5
7352/7352 [==============================] - 6s 806us/sample - loss: 0.7920 - acc: 0.6726 - val_loss: 0.5194 - val_acc: 0.8120
Epoch 2/40
7232/7352 [============================>.] - ETA: 0s - loss: 0.3321 - acc: 0.8760
Epoch 00002: val_acc improved from 0.81201 to 0.87479, saving model to 1D_CNN_Best.hdf5
7352/7352 [==============================] - 2s 254us/sample - loss: 0.3316 - acc: 0.8757 - val_loss: 0.3728 - val_acc: 0.8748
Epoch 3/40
7232/7352 [============================>.] - ETA: 0s - loss: 0.2212 - acc: 0.9206
Epoch 00003: val_acc improved from 0.87479 to 0.88802, saving model to 1D_CNN_Best.hdf5
7352/7352 [==============================] - 2s 312us/sample - loss: 0.2205 - acc: 0.9207 - val_loss: 0.3517 - val_acc: 0.8880
Epoch 4/40
7296/7352 [=========

Epoch 30/40
7040/7352 [===========================>..] - ETA: 0s - loss: 0.0751 - acc: 0.9618
Epoch 00030: val_acc did not improve from 0.93858
7352/7352 [==============================] - 2s 207us/sample - loss: 0.0748 - acc: 0.9618 - val_loss: 0.4852 - val_acc: 0.9158
Epoch 31/40
7104/7352 [===========================>..] - ETA: 0s - loss: 0.0890 - acc: 0.9586
Epoch 00031: val_acc did not improve from 0.93858
7352/7352 [==============================] - 2s 221us/sample - loss: 0.0880 - acc: 0.9592 - val_loss: 0.4797 - val_acc: 0.9203
Epoch 32/40
7168/7352 [============================>.] - ETA: 0s - loss: 0.0712 - acc: 0.9622
Epoch 00032: val_acc did not improve from 0.93858
7352/7352 [==============================] - 2s 209us/sample - loss: 0.0703 - acc: 0.9626 - val_loss: 0.5319 - val_acc: 0.9230
Epoch 33/40
7104/7352 [===========================>..] - ETA: 0s - loss: 0.0752 - acc: 0.9635
Epoch 00033: val_acc did not improve from 0.93858
7352/7352 [==============================] 

In [107]:
# Re-evaluate the model
model.load_weights(filepath)

loss, acc = model.evaluate(testX, testy, verbose=2)
print("1D CNN model, accuracy: {:5.2f}%".format(100 * acc))

2947/2947 - 0s - loss: 0.3960 - acc: 0.9386
1D CNN model, accuracy: 93.86%


# 2D Convolutions

In [119]:
trainX, trainy, testX, testy = load_dataset()

trainX = trainX.reshape(7352,trainX.shape[1], trainX.shape[2], 1)
print(trainX.shape)

testX = testX.reshape(2947, testX.shape[1], testX.shape[2], 1)
print(testX.shape)

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
(7352, 128, 9, 1)
(2947, 128, 9, 1)


In [109]:
# 2-D CNN Network
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2], 1)))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 124, 5, 64)        1664      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 61, 2, 32)         18464     
_________________________________________________________________
flatten_12 (Flatten)         (None, 3904)              0         
_________________________________________________________________
dense_60 (Dense)             (None, 32)                124960    
_________________________________________________________________
dropout_17 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 6)                 198       
Total params: 145,286
Trainable params: 145,286
Non-trainable params: 0
_______________________________________________

In [113]:
filepath = '2D_CNN_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, nb_epoch=40,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/40
7232/7352 [============================>.] - ETA: 0s - loss: 0.0743 - acc: 0.9616
Epoch 00001: val_acc did not improve from 0.92942
7352/7352 [==============================] - 4s 525us/sample - loss: 0.0746 - acc: 0.9611 - val_loss: 0.3359 - val_acc: 0.9233
Epoch 2/40
7232/7352 [============================>.] - ETA: 0s - loss: 0.0749 - acc: 0.9643
Epoch 00002: val_acc did not improve from 0.92942
7352/7352 [==============================] - 4s 569us/sample - loss: 0.0753 - acc: 0.9641 - val_loss: 0.4221 - val_acc: 0.9223
Epoch 3/40
7232/7352 [============================>.] - ETA: 0s - loss: 0.0722 - acc: 0.9670
Epoch 00003: val_acc did not improve from 0.92942
7352/7352 [==============================] - 4s 601us/sample - loss: 0.0741 - acc: 0.9669 - val_loss: 0.5645 - val_acc: 0.9148
Epoch 4/40
7232/7352 [============================>.] - ETA: 0s - loss: 0.0808 - acc: 0.9640
Epoch 00004: val_acc did not improve from 0.92942

Epoch 31/40
7296/7352 [============================>.] - ETA: 0s - loss: 0.0332 - acc: 0.9868
Epoch 00031: val_acc did not improve from 0.94062
7352/7352 [==============================] - 5s 739us/sample - loss: 0.0330 - acc: 0.9869 - val_loss: 0.8296 - val_acc: 0.9253
Epoch 32/40
7296/7352 [============================>.] - ETA: 0s - loss: 0.0378 - acc: 0.9840
Epoch 00032: val_acc did not improve from 0.94062
7352/7352 [==============================] - 5s 728us/sample - loss: 0.0379 - acc: 0.9839 - val_loss: 0.6082 - val_acc: 0.9304
Epoch 33/40
7232/7352 [============================>.] - ETA: 0s - loss: 0.0374 - acc: 0.9835
Epoch 00033: val_acc did not improve from 0.94062
7352/7352 [==============================] - 5s 727us/sample - loss: 0.0378 - acc: 0.9831 - val_loss: 0.7657 - val_acc: 0.9332
Epoch 34/40
7232/7352 [============================>.] - ETA: 0s - loss: 0.0320 - acc: 0.9874
Epoch 00034: val_acc did not improve from 0.94062
7352/7352 [==============================] 

In [120]:
model.load_weights(filepath)
loss, acc = model.evaluate(testX, testy, verbose=2)
print("2D CNN model, accuracy: {:5.2f}%".format(100 * acc))

2947/2947 - 0s - loss: 0.6234 - acc: 0.9423
2D CNN model, accuracy: 94.23%


# Using only ACC and GYRO

In [126]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        #print(data.shape)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [127]:
# Load the data
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 6) (7352, 6) (2947, 128, 6) (2947, 6)


In [128]:
# 1-D CNN Network
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=2, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_17 (Conv1D)           (None, 124, 64)           1984      
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 61, 32)            6176      
_________________________________________________________________
flatten_14 (Flatten)         (None, 1952)              0         
_________________________________________________________________
dense_64 (Dense)             (None, 32)                62496     
_________________________________________________________________
dropout_19 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 6)                 198       
Total params: 70,854
Trainable params: 70,854
Non-trainable params: 0
_________________________________________________

In [129]:
filepath = '1D_CNN_Best_2.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, nb_epoch=100,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Epoch 1/100
7232/7352 [============================>.] - ETA: 0s - loss: 1.1814 - acc: 0.4634
Epoch 00001: val_acc improved from -inf to 0.63488, saving model to 1D_CNN_Best_2.hdf5
7352/7352 [==============================] - 3s 390us/sample - loss: 1.1747 - acc: 0.4653 - val_loss: 0.7236 - val_acc: 0.6349
Epoch 2/100
6976/7352 [===========================>..] - ETA: 0s - loss: 0.6879 - acc: 0.6380
Epoch 00002: val_acc improved from 0.63488 to 0.64065, saving model to 1D_CNN_Best_2.hdf5
7352/7352 [==============================] - 1s 150us/sample - loss: 0.6851 - acc: 0.6396 - val_loss: 0.6841 - val_acc: 0.6407
Epoch 3/100
6976/7352 [===========================>..] - ETA: 0s - loss: 0.6285 - acc: 0.6544
Epoch 00003: val_acc improved from 0.64065 to 0.68035, saving model to 1D_CNN_Best_2.hdf5
7352/7352 [==============================] - 1s 152us/sample - loss: 0.6266 - acc: 0.6552 - val_loss: 0.6179 - val_acc: 0.6804
Epoch 4/100
7168/7352 

Epoch 29/100
7168/7352 [============================>.] - ETA: 0s - loss: 0.1725 - acc: 0.9342
Epoch 00029: val_acc did not improve from 0.91381
7352/7352 [==============================] - 1s 148us/sample - loss: 0.1731 - acc: 0.9340 - val_loss: 0.3034 - val_acc: 0.9057
Epoch 30/100
6976/7352 [===========================>..] - ETA: 0s - loss: 0.1633 - acc: 0.9399
Epoch 00030: val_acc improved from 0.91381 to 0.91449, saving model to 1D_CNN_Best_2.hdf5
7352/7352 [==============================] - 1s 155us/sample - loss: 0.1653 - acc: 0.9392 - val_loss: 0.3269 - val_acc: 0.9145
Epoch 31/100
7232/7352 [============================>.] - ETA: 0s - loss: 0.1627 - acc: 0.9390
Epoch 00031: val_acc did not improve from 0.91449
7352/7352 [==============================] - 1s 150us/sample - loss: 0.1634 - acc: 0.9392 - val_loss: 0.3390 - val_acc: 0.9125
Epoch 32/100
7040/7352 [===========================>..] - ETA: 0s - loss: 0.1856 - acc: 0.9305
Epoch 00032: val_acc improved from 0.91449 to 0.9

Epoch 59/100
7040/7352 [===========================>..] - ETA: 0s - loss: 0.0931 - acc: 0.9662
Epoch 00059: val_acc did not improve from 0.92060
7352/7352 [==============================] - 1s 157us/sample - loss: 0.0963 - acc: 0.9650 - val_loss: 0.4089 - val_acc: 0.9155
Epoch 60/100
7296/7352 [============================>.] - ETA: 0s - loss: 0.0978 - acc: 0.9635- ETA: 0s - loss: 0.0967 - acc
Epoch 00060: val_acc did not improve from 0.92060
7352/7352 [==============================] - 1s 151us/sample - loss: 0.0978 - acc: 0.9635 - val_loss: 0.4705 - val_acc: 0.9135
Epoch 61/100
7040/7352 [===========================>..] - ETA: 0s - loss: 0.1011 - acc: 0.9643
Epoch 00061: val_acc did not improve from 0.92060
7352/7352 [==============================] - 1s 146us/sample - loss: 0.1012 - acc: 0.9642 - val_loss: 0.4890 - val_acc: 0.9019
Epoch 62/100
7104/7352 [===========================>..] - ETA: 0s - loss: 0.1019 - acc: 0.9631
Epoch 00062: val_acc did not improve from 0.92060
7352/7352

Epoch 89/100
7168/7352 [============================>.] - ETA: 0s - loss: 0.0598 - acc: 0.9791
Epoch 00089: val_acc did not improve from 0.92094
7352/7352 [==============================] - 3s 415us/sample - loss: 0.0594 - acc: 0.9793 - val_loss: 0.5704 - val_acc: 0.9172
Epoch 90/100
7232/7352 [============================>.] - ETA: 0s - loss: 0.0532 - acc: 0.9816
Epoch 00090: val_acc did not improve from 0.92094
7352/7352 [==============================] - 3s 415us/sample - loss: 0.0529 - acc: 0.9818 - val_loss: 0.6442 - val_acc: 0.8999
Epoch 91/100
7232/7352 [============================>.] - ETA: 0s - loss: 0.0656 - acc: 0.9747
Epoch 00091: val_acc did not improve from 0.92094
7352/7352 [==============================] - 3s 366us/sample - loss: 0.0653 - acc: 0.9747 - val_loss: 0.6187 - val_acc: 0.9128
Epoch 92/100
7168/7352 [============================>.] - ETA: 0s - loss: 0.0550 - acc: 0.9795
Epoch 00092: val_acc did not improve from 0.92094
7352/7352 [============================

In [130]:
# Re-evaluate the model
model.load_weights(filepath)

loss, acc = model.evaluate(testX, testy, verbose=2)
print("1D CNN model, accuracy: {:5.2f}%".format(100 * acc))

2947/2947 - 0s - loss: 0.4455 - acc: 0.9209
1D CNN model, accuracy: 92.09%


# Using LSTM

In [131]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values
 
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        #print(data.shape)
        loaded.append(data)
        
    loaded = dstack(loaded)
    return loaded

# load train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'datasets/UCI-HAR-Dataset/')
    
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy



In [134]:
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


In [135]:
# LSTM Network Default input: [batch, timesteps, feature]
import tensorflow as tf

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(trainX.shape[1], trainX.shape[2])))
model.add(tf.keras.layers.LSTM(16, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(trainy.shape[1], activation='softmax'))
#model.output_shape

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 16)                1664      
_________________________________________________________________
dense_68 (Dense)             (None, 32)                544       
_________________________________________________________________
dropout_21 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 6)                 198       
Total params: 2,406
Trainable params: 2,406
Non-trainable params: 0
_________________________________________________________________


In [136]:
filepath = 'LSTM_Best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, 
                             monitor='val_acc',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')

callbacks = [checkpoint]

model.fit(trainX, trainy, batch_size=64, nb_epoch=40,
            verbose=1, validation_data=(testX, testy),
         callbacks = callbacks)

Train on 7352 samples, validate on 2947 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/40
7296/7352 [============================>.] - ETA: 0s - loss: 2.4648 - acc: 0.3768
Epoch 00001: val_acc improved from -inf to 0.54801, saving model to LSTM_Best.hdf5
7352/7352 [==============================] - 10s 1ms/sample - loss: 2.4560 - acc: 0.3789 - val_loss: 1.7507 - val_acc: 0.5480
Epoch 2/40
7296/7352 [============================>.] - ETA: 0s - loss: 2441.6731 - acc: 0.5228
Epoch 00002: val_acc improved from 0.54801 to 0.56295, saving model to LSTM_Best.hdf5
7352/7352 [==============================] - 8s 1ms/sample - loss: 2423.0853 - acc: 0.5227 - val_loss: 1.3928 - val_acc: 0.5629
Epoch 3/40
7296/7352 [============================>.] - ETA: 0s - loss: 1.4325 - acc: 0.5513
Epoch 00003: val_acc did not improve from 0.56295
7352/7352 [==============================] - 9s 1ms/sample - loss: 1.4330 - acc: 0.5509 - val_loss: 1.3

Epoch 28/40
7296/7352 [============================>.] - ETA: 0s - loss: 0.7918 - acc: 0.6512
Epoch 00028: val_acc improved from 0.60299 to 0.60333, saving model to LSTM_Best.hdf5
7352/7352 [==============================] - 15s 2ms/sample - loss: 0.7915 - acc: 0.6513 - val_loss: 0.8691 - val_acc: 0.6033
Epoch 29/40
7296/7352 [============================>.] - ETA: 0s - loss: 0.7829 - acc: 0.6573
Epoch 00029: val_acc improved from 0.60333 to 0.61351, saving model to LSTM_Best.hdf5
7352/7352 [==============================] - 13s 2ms/sample - loss: 0.7828 - acc: 0.6574 - val_loss: 0.8651 - val_acc: 0.6135
Epoch 30/40
7296/7352 [============================>.] - ETA: 0s - loss: 0.7789 - acc: 0.6497
Epoch 00030: val_acc improved from 0.61351 to 0.61384, saving model to LSTM_Best.hdf5
7352/7352 [==============================] - 12s 2ms/sample - loss: 0.7797 - acc: 0.6489 - val_loss: 0.8602 - val_acc: 0.6138
Epoch 31/40
7296/7352 [============================>.] - ETA: 0s - loss: 0.7733 - 

KeyboardInterrupt: 

# Using BI-Directional LSTM